## Initialise RAG

In [6]:

from RAG.librarian import Librarian 

librarian = Librarian(librarian_LLM_model = "GEMINI")

# SELECT SPECIALIST DATABASE
librarian.select_specialist(specialist = "traveller", specialist_LLM_model = "GEMINI", )

# Ask librarian to get acquinted with the specialist database
librarian.Traveller.load_data_model(reembed = False,
                                    embed_id = 0,
                                    data_model_keys = {"TEST - CLIENT":"CLIENT ID",
                                                        "TEST - CLIENT REQUEST":"CLIENT ID",
                                                        "TEST - FLIGHTS":"FLIGHT ID",
                                                        "TEST - ACCOMODATIONS":"ACCOMODATION ID",
                                                        "TEST - ACTIVITIES":"ACTIVITY ID",
                                                        "TEST - SERVICES":"SERVICE ID",
                                                        },
                                    reembed_table = {"TEST - CLIENT":True,
                                                    "TEST - CLIENT REQUEST":True,
                                                    "TEST - FLIGHTS":True,
                                                    "TEST - ACCOMODATIONS":True,
                                                    "TEST - ACTIVITIES":True,
                                                    "TEST - SERVICES":True,
                                                    }
                                                    
                                    )



KeyboardInterrupt



## _*Generate*_ : travel package from customer/agent prompt  + inventory

In [ ]:
# Ask Traveller to generate a travel package
initial_query = "Hafeez from Kuala Lumpur wants to go Bali for 4 days for 2 pax. Budget: $2000. I want to chill at the beach"

convo_package = librarian.Traveller.III_generate_travel_package(initial_query = initial_query,
                                                                 topN = 6, 
                                                                 model_name = "gemini-pro",
                                                                 )

Token size of the prompt for cl100k_base ~ 3745
**Summary**

Immerse yourself in the beauty of Bali on this 4-day beach getaway designed to soothe your soul and rejuvenate your senses. Dive into the pristine waters, bask in the warm sun, indulge in luxurious spa treatments, and explore the island's stunning natural wonders. This trip_recommendation offers a perfect blend of relaxation, adventure, and cultural experiences, ensuring an unforgettable escape to paradise.

**Journey Highlights**

* Chill out on the white-sand beaches of Bali
* Pamper yourself with a rejuvenating spa treatment
* Embark on a scenic hiking and cliff-hunting tour
* Savor delicious meals at local restaurants
* Capture the beauty of Bali's rice fields
* Your journey takes you to: Kuala Lumpur - Denpasar Bali - Kuala Lumpur

**Itinerary**

**Day 1**

* 0900hrs: Depart from Kuala Lumpur International Airport (FLIGHT ID: 1)
* 1100hrs: Arrive at Denpasar Bali Airport (DPS)
* 1200hrs: Transfer to Hard Rock Hotel Bali 

## Regenerate : travel package from followup prompt + inventory

In [ ]:
followup_query = "Give me the cheapest bali activities instead of that 4.5K hunting tour"

convo_package = librarian.Traveller.III_generate_travel_package(initial_query = "",
                                                                followup_query = followup_query,
                                                                 topN = 6, 
                                                                 model_name = "gemini-pro",
                                                                 )

Token size of the prompt for cl100k_base ~ 3857
**Refined Travel Package**

**Summary**

Immerse yourself in the beauty of Bali on this tailored 4-day beach getaway designed to soothe your soul and rejuvenate your senses. Dive into the pristine waters, bask in the warm sun, indulge in luxurious spa treatments, and explore the island's stunning natural wonders. This trip offers a perfect blend of relaxation, adventure, and cultural experiences, ensuring an unforgettable escape to paradise.

**Journey Highlights**

* Chill out on the white-sand beaches of Bali
* Pamper yourself with a rejuvenating spa treatment
* Embark on a scenic hiking and cliff-hunting tour
* Savor delicious meals at local restaurants
* Capture the beauty of Bali's rice fields
* Your journey takes you to: Kuala Lumpur - Denpasar Bali - Kuala Lumpur

**Itinerary**

**Day 1**

* 0900hrs: Depart from Kuala Lumpur International Airport (FLIGHT ID: 1)
* 1100hrs: Arrive at Denpasar Bali Airport (DPS)
* 1200hrs: Transfer to

## OCR : extract text from image

### a) EasyOCR(image: jpg)

In [ ]:
import easyocr

def extract_text_from_image(image_file):
  """
  Extracts text and calculates average probability from an image using EasyOCR.

  Args:
      image_file: Path to the image file.

  Returns:
      A tuple containing:
          * Extracted text as a single string concatenated from all blocks.
          * Raw output from EasyOCR (list of tuples with text, probability, and bounding box).
          * Average probability of all text blocks.
  """

  # Create a reader for the languages you want to support
  reader = easyocr.Reader(['en'])

  # Use the reader to read the text from the image
  raw = reader.readtext(image_file)

  # Extract text and probabilities
  text = ' '.join([block[1] for block in raw])
  probabilities = [block[2] for block in raw]

  # Calculate average probability (handling potential division by zero)
  average_probability = sum(probabilities) / len(probabilities) if probabilities else 0.0

  return text, raw, average_probability

# Example usage
# image_file = './database/travel/Ingest/Cust_req_1.jpg'
image_path = './database/travel/Ingest/TP_Cxxx_20240105_KL1.jpg'
text, raw, avg_prob = extract_text_from_image(image_path)
print("Extracted Text:", text)
# print("Raw Data:", raw)
print("Average Probability:", avg_prob)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Extracted Text: exil Ieo @ su-nig {6v.m4 One- no } | 03 2114 5414 Ajeat 54nia Trave( "V^+ I(tuc SMART Travel Agency :Sedunia Travel Services WeRLD Package Kuala Lumpur Tour Date 05 January 2024 Kuala Lumpur Tour No. of Pax 04 pax (02 adult & 02 child) Date 05 January 2024 11.OOAM 7 OOPM Full Day (8 Hours) Time Pick up Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kualo Lumpur, Drop off Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kuala Lumpur, Type of Mini Van Vehicle Tour Guide English Speaking Entrance Fee Not included Meals Adult (MYR 50.00) Child (MYR 25.00) etarian Meal for Indian family (MYR) MYR 550.00 (transport/tour) MYR 300 (Tour guide) MYR 300 (meals) MYR3OO Price (markup) Highlight Gaze at the beautiful city of Kuala Lumpur from the Observation Deck of the Petronas Twin Towers Climb Batu Caves to explore Southeast Asia'$ most visited Hindu Temple Capture your

In [ ]:
text

'exil Ieo @ su-nig {6v.m4 One- no } | 03 2114 5414 Ajeat 54nia Trave( "V^+ I(tuc SMART Travel Agency :Sedunia Travel Services WeRLD Package Kuala Lumpur Tour Date 05 January 2024 Kuala Lumpur Tour No. of Pax 04 pax (02 adult & 02 child) Date 05 January 2024 11.OOAM 7 OOPM Full Day (8 Hours) Time Pick up Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kualo Lumpur, Drop off Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kuala Lumpur, Type of Mini Van Vehicle Tour Guide English Speaking Entrance Fee Not included Meals Adult (MYR 50.00) Child (MYR 25.00) etarian Meal for Indian family (MYR) MYR 550.00 (transport/tour) MYR 300 (Tour guide) MYR 300 (meals) MYR3OO Price (markup) Highlight Gaze at the beautiful city of Kuala Lumpur from the Observation Deck of the Petronas Twin Towers Climb Batu Caves to explore Southeast Asia\'$ most visited Hindu Temple Capture your memories with

### b) LLM(image: jpg)

In [4]:
image_path = './database/travel/Ingest/partners - johor - theme park_legoland.pdf'
prompt_1 = """The image is a travel itinerary that i want to digitise so that i can populate a database. 
Help analyse the content and reproduce it in a structured format such that it is easily extractable
"""
from RAG.librarian import Librarian
librarian = Librarian(librarian_LLM_model = "GEMINI")
# SELECT SPECIALIST DATABASE
librarian.select_specialist(specialist = "traveller", specialist_LLM_model = "GEMINI", )

# Ask librarian to get acquinted with the specialist database
librarian.Traveller.load_data_model(reembed = False,
                                    embed_id = 0,
                                    data_model_keys = {"TEST - CLIENT":"CLIENT ID",
                                                        "TEST - CLIENT REQUEST":"CLIENT ID",
                                                        "TEST - FLIGHTS":"FLIGHT ID",
                                                        "TEST - ACCOMODATIONS":"ACCOMODATION ID",
                                                        "TEST - ACTIVITIES":"ACTIVITY ID",
                                                        "TEST - SERVICES":"SERVICE ID",
                                                        },
                                    reembed_table = {"TEST - CLIENT":True,
                                                    "TEST - CLIENT REQUEST":True,
                                                    "TEST - FLIGHTS":True,
                                                    "TEST - ACCOMODATIONS":True,
                                                    "TEST - ACTIVITIES":True,
                                                    "TEST - SERVICES":True,
                                                    }
                                                    
                                    )
response = librarian.Traveller.model_specialist.prompt_image(image_path = image_path,
                                                  prompt_1 = prompt_1,
                                                  prompt_2 = None,
                                                  model_name = "gemini-pro-vision",)

loading specialist: TRAVELLER ...
TRAVELLER embedding: TEST - CLIENT - LOADED
TRAVELLER embedding: TEST - CLIENT REQUEST - LOADED
TRAVELLER embedding: TEST - FLIGHTS - LOADED
TRAVELLER embedding: TEST - ACCOMODATIONS - LOADED
TRAVELLER embedding: TEST - ACTIVITIES - LOADED
TRAVELLER embedding: TEST - SERVICES - LOADED
TRAVELLER loaded


FileNotFoundError: Could not find image: database\travel\Ingest\partners - johor - theme park_legoland.pdf

In [16]:
import PyPDF2

def extract_text_from_pdf(pdf_file):
    """
    Extracts text from a PDF file using PyPDF2.
    
    Args:
        pdf_file: Path to the PDF file.
    
    Returns:
        Extracted text as a single string.
    """
    
    # Open the PDF file in read-binary mode
    with open(pdf_file, 'rb') as file:
        # Create a PDF file reader
        reader = PyPDF2.PdfReader(file)
        # Initialize an empty string to store the text
        text = ''
        # Loop through all pages in the PDF
        for page_num in range(len(reader.pages)):
            # Get the page
            page = reader.pages[page_num]
            # Extract the text from the page
            i = page.extract_text()
            text += i
            print(i)
    return text

# Example usage
pdf_file = './database/Ingest/partners - johor - theme park_legoland.pdf'
text = extract_text_from_pdf(pdf_file)

In [19]:
pip install -r requirements.txt


  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gpsoauth 1.1.0 requires urllib3<2.0, but you have urllib3 2.2.1 which is incompatible.


In [1]:
# import pytesseract

from pdf2image import convert_from_path
import os

from settings import GEMINI_API_KEY
from llm_handler.GHandler import GHandler

ghandler = GHandler(GEMINI_API_KEY,                  
                    generation_config = {"temperature": 0.9,
                                        "top_p": 0.95,
                                        "top_k": 40,
                                        "max_output_tokens": 40000,
                                        },
                    block_threshold="BLOCK_NONE",
                    )

prompt_1 = "You are an OCR bot. Extract ALL the text from the image as raw text. Ensure all pricing, phone numbers and emails are extracted ACCURATELY. OCR text output is sometimes wrong, so correct it where needed."
# Specify the directory containing the PDF files
pdf_dir = './downloaded_pdfs/partners - malacca - attractions'#partners - johor - theme park_legoland.pdf'

# Loop through all files in the directory
for filename in os.listdir(pdf_dir):
    print(filename)
    if filename.endswith('.pdf'):
        # Open the PDF file
        print('processing:', filename)
        pdf_file = os.path.join(pdf_dir, filename)
        
        # Convert PDF to image
        images = convert_from_path(pdf_file)
        responses = []
        for image in images:
            # image.show()
            # save image to file
            image_path = './database/Ingest/' + filename.replace('.pdf', '.jpg')
            image.save(image_path, 'JPEG')
            print('processing:', image_path)
            response = ghandler.prompt_image(image_path = image_path,
                                                  prompt_1 = prompt_1,
                                                  prompt_2 = None,
                                                  model_name = "gemini-pro-vision",)
            print(response)
            responses.append(response.text)
            # use ghandler image analyses

d:\PycharmProjects\smart-travels\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


menara taming sari.pdf
processing: menara taming sari.pdf


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [7]:
pip install PyMuPDF

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
    --------------------------------------- 0.0/3.2 MB 991.0 kB/s eta 0:00:04
   ----------------- ---------------------- 1.4/3.2 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------  3.2/3.2 MB 25.6 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 22.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
   ------- -------------------------------- 2.3/12.4 MB 48.8 MB/s eta 0:00:01
   --------------- ------------------------ 4.7/12.4 MB 50.3 MB/s eta 0:00:01
   --------------------- ------------------ 6.7/12.4 MB 48.1 MB/s eta 0:00:01
   -------------------------- ------------- 8.4/12.4 MB 44.8 MB/s eta 0:00:01
   ---------------------------------- ----- 10.8/12.4 MB 46.7 MB/s eta 0:00:01
   ---------------------------------------  12.4/12.4 MB 46.7 MB/s eta 0:00:01
   ---------------------------------------- 12.4/12.4 MB 40.9 MB/s eta 0:00:00
Not

In [10]:
import fitz  # Import PyMuPDF
pdf_file = './downloaded_pdfs/partners - malacca - attractions/menara taming sari.pdf' #'./database/Ingest/partners - johor - theme park_legoland.pdf'
def gemini_ocr(pdf_file):
    """Performs OCR on the given PDF using Gemini and returns extracted text."""
    ghandler = GHandler(GEMINI_API_KEY, generation_config={"temperature": 0.9, "top_p": 0.95, "top_k": 40, "max_output_tokens": 40000}, block_threshold="BLOCK_NONE")
    prompt = "You are an OCR bot. Extract ALL the text from the image as raw text. Ensure all pricing, phone numbers, and emails are extracted ACCURATELY. OCR text output is sometimes wrong, so correct it where needed."

    doc = fitz.open(pdf_file)
    extracted_text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))  # Increase resolution for better OCR
        image_path = pdf_file.replace('.pdf', f'_page_{page_num + 1}.jpg')
        pix.save(image_path)  # Save each page as an image

        response = ghandler.prompt_image(image_path=image_path, prompt_1=prompt, prompt_2=None, model_name="gemini-pro-vision")
        extracted_text += response.text
        os.remove(image_path)  # Clean up the image file

    return extracted_text

In [11]:
test = gemini_ocr(pdf_file)

In [13]:
print(test)

 MENARA TAMING SARI
澄沙里塔
தாமிரிசாரி கோபுரம்
TAMING SARI TOWER
MELAKA

Taming Sari Tower
Bird's-Eye View of Melaka
www.menaratamingsari.com

4th Tallest Revolving Tower In Malaysia
Revolving at 90 meters above sea level
360-degree view of the historical city of Melaka

MENARA TAMING SARI SDN. BHD. (744024-H)
Wisma Taming Sari, Jalan Merdeka, 75000 Melaka, Malaysia.
Tel: 06-281 6888 Fax: 06-281 6066
Email: ticketing@menaratamingsari.com

Adult: RM50.00
Child: RM25.00
Family Package (2 adults + 2 children): RM100.00
(All prices are inclusive of 6% Service Tax) A Breathtaking View From Above

A 360° viewing tower that provides a breath-taking view from a height of 80 meters in the air. The 7-minute ride enables you to view spectacular and panoramic sights of Melaka, UNESCO World Heritage City such as St. Paul's Hill, Independence Memorial Building, Samudra Museum (Flor De La Mar), Melaka River, Dataran Pahlawan, Selat Melaka Mosque, Pulau Besar and the Straits of Melaka.

While enjoying th

In [6]:
pip install -c conda-forge poppler

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'conda-forge'


In [18]:
print(len(responses[1]))
print(responses[1])
#  count number of characters in the text

1280
 MATTA FAIR EXCLUSIVE
2D1N Room with Breakfast @ LEGOLAND® Hotel
Stay Period: 22 Mar - 30 Sept 2024

* Each room sleeps up to 5 persons (2 adults & 3 children or 3 adults & 2 children)
* Offer valid for Themed Room and Premium Themed Room category. Applicable for Pirate, Kingdom and Adventure Theme

Themed Room 
UP: RM950
RM650 nett** per night

Premium Themed Room 
UP: RM1,100
RM750 nett** per night

**Surcharge apply: RM50 (Fridays & Saturdays)
RM150 (24 May - 23 June, 15 Jul - 23 Aug, 31 Aug - 8 Sept, 13 - 21 Sept 2024)

STAY & PLAY PACKAGE FOR A FAMILY OF 4 FROM RM1,762**
Save up to RM500

2D1N** @ LEGOLAND® Hotel
Enjoy a unique LEGO® overnight experience

BREAKFAST at Bricks Family Restaurant
Buffet breakfast in a child-friendly dining environment for up to 5 pax

TRIPLE-PARK ANNUAL PASS*
2 Adults & 2 Children
1 year unlimited access to theme park, Water Park and Aquarium

FREE RM100 cash voucher

*Collection of Annual Pass voucher on the date of arrival. **Price based on 2 A

# 2) FM RAG

In [1]:
from RAG.cohere_RAG import RAG
query = "i want to go to a perlis cultural tour, give me a 3 days itinerary"
rag = RAG()
result = rag.rag_pipeline(query = query, sheet_name = "Master Database", worksheet_name="inventory", reembed= True)

In [2]:
print(result)

Here's a 3-day cultural itinerary in Perlis, tailored to your request. 

## Day 1: 
**Perlis Cycling Tour**
- Price: RM339
- Vendor: unicastana holidays
- Activities: Cycling and exploring local night markets and night activities.

## Day 2: 
**Touch The Sky**
- Price: RM499
- Vendor: unicastana holidays
- Activities: View sunrise, caving, hiking and bird watching. 

## Day 3: 
**Green Paradise**
- Price: RM479
- Vendor: unicastana holidays
- Activities: Explore the nature of the millions of years old limestone cave, known as Gua Wang Burma or Gua Wang Buluh and other scenic spots like the Jungle Trekking, for a view of the sunset.

Other recommended attractions include the following:
- Gua Gendah
- Gua Kelam 1 & 2
- Gua Wang Burma 1 & 2
- Menara Pandang Indah
- Bukit Keteri
- Tasoh Lake
- Kuala Perlis Coast


In [ ]:
pip install llama-index

   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   - -------------------------------------- 0.4/15.4 MB 13.9 MB/s eta 0:00:02
   --- ------------------------------------ 1.2/15.4 MB 14.7 MB/s eta 0:00:01
   ----- ---------------------------------- 2.1/15.4 MB 15.0 MB/s eta 0:00:01
   -------- ------------------------------- 3.3/15.4 MB 17.4 MB/s eta 0:00:01
   ------------ --------------------------- 4.8/15.4 MB 20.3 MB/s eta 0:00:01
   ---------------- ----------------------- 6.5/15.4 MB 23.0 MB/s eta 0:00:01
   --------------------- ------------------ 8.2/15.4 MB 24.8 MB/s eta 0:00:01
   ------------------------- -------------- 9.9/15.4 MB 27.6 MB/s eta 0:00:01
   ------------------------------- -------- 11.9/15.4 MB 32.8 MB/s eta 0:00:01
   ------------------------------------ --- 14.0/15.4 MB 38.6 MB/s eta 0:00:01
   ---------------------------------------  15.4/15.4 MB 38.5 MB/s eta 0:00:01
   ---------------------------------------  15.4/15.4 MB 38.5 MB/s et

In [2]:
pip install llama-index-readers-google

     ---------------------------------------- 0.0/987.4 kB ? eta -:--:--
     ---------------------------------------- 10.2/987.4 kB ? eta -:--:--
     - ----------------------------------- 51.2/987.4 kB 650.2 kB/s eta 0:00:02
     ---------------------- --------------- 593.9/987.4 kB 5.4 MB/s eta 0:00:01
     -------------------------------------- 987.4/987.4 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/48.9 kB ? eta -:--:--
     ---------------------------------------- 48.9/48.9 kB 2.6 MB/s eta 0:00:00
   ----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
types-requests 2.31.0.20240406 requires urllib3>=2, but you have urllib3 1.26.18 which is incompatible.


In [4]:
from llama_index.readers.google import GoogleDriveReader
loader = GoogleDriveReader(credentials_path='./gdrive/phrasal-ability-419201-d527372ace3b.json')

In [7]:
loader.load_data()

ValueError: Client secrets must be for a web or installed app.

In [3]:
import gspread
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2 import service_account


class GspreadHandler:
    def __init__(self, credentials_filepath):
        self.credentials_filepath = credentials_filepath

        # Initialize both gspread (for sheets) and Drive API (for other files)
        self.gc = gspread.service_account(filename=self.credentials_filepath)
        self.creds = service_account.Credentials.from_service_account_file(
            self.credentials_filepath, scopes=['https://www.googleapis.com/auth/drive']
        )
        self.drive_service = build('drive', 'v3', credentials=self.creds)

    # ... (Your existing sheet-related methods remain unchanged) ...

    def get_file_content(self, file_id):
        """Downloads and returns the content of a file from Google Drive.

        Args:
            file_id (str): The ID of the file in Google Drive.

        Returns:
            bytes: The content of the file (exact format depends on file type).
        """
        request = self.drive_service.files().get_media(fileId=file_id)
        file_content = request.execute()
        return file_content

    def list_files(self, query=None):
        """Lists files in Google Drive based on an optional query.

        Args:
            query (str, optional): A search query to filter the results (see Drive API docs).

        Returns:
            list: A list of file metadata dictionaries (name, id, etc.).
        """
        results = self.drive_service.files().list(q=query).execute()
        items = results.get('files', [])
        return items


In [4]:
handler = GspreadHandler('your_credentials.json')

# Get a spreadsheet as a DataFrame
df = handler.get_sheet_as_df("YourSheetName", "YourWorksheetName")
print(df)

# List files in a specific folder
folder_id = 'your_folder_id'
files = handler.list_files(f"'{folder_id}' in parents")
for file in files:
    print(f"File Name: {file['name']}, ID: {file['id']}")

# Download and process a file
file_id = 'your_file_id'
file_content = handler.get_file_content(file_id)
# ... (Handle the file content based on its type, e.g., CSV, image, etc.)


FileNotFoundError: [Errno 2] No such file or directory: 'your_credentials.json'

# GCP drive handler

In [5]:
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install PyDrive2

  Using cached cffi-1.16.0-cp312-cp312-win_amd64.whl.metadata (1.5 kB)
   ---------------------------------------- 0.0/46.7 kB ? eta -:--:--
   ---------------------------------------- 46.7/46.7 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/56.9 kB ? eta -:--:--
   ---------------------------------------- 56.9/56.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------------ --------------------------- 0.9/2.9 MB 28.4 MB/s eta 0:00:01
   ---------------------------------- ----- 2.5/2.9 MB 31.8 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 30.6 MB/s eta 0:00:00
Using cached cffi-1.16.0-cp312-cp312-win_amd64.whl (181 kB)
   ---------------------------------------- 0.0/117.6 kB ? eta -:--:--
   ---------------------------------------- 117.6/117.6 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## 1) Oauth 

In [21]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

gauth = GoogleAuth()
gauth.settings['client_config_file'] = r'./client_secrets.json'
gauth.LocalWebserverAuth()

drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=995857605983-onp1ieqbr9dr12ivrui5igins8trst4d.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=online&response_type=code

Authentication successful.


In [23]:
# Auto-iterate through all files that matches this query
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: SW Backend Docs, id: 1U2jWotVN5a3AKoPCt7a-9j-Y3dTj8bmF
title: Consultant Agreement - CL and Shaik 16 May 2024, id: 1s0julYLUV0Slyv0nsWIVtg-NvKt-oU_SAqAGt7F5CSo
title: give me the arabic script for the following dua i..., id: 1Tz_9NvMw0cEHq7MJe6CP7-cM9SKGCxS70yj8fzOWm5w
title: Meeting Summary Proposal, id: 1H22V48Pz6a49umVnboyEwspW9Hfh0Ce9fQWhtJIB8MQ
title: Freedom Phoenix Inc x AI Proposal , id: 1dNP4G0O3QyVduLIDerJwmt3lq8oWUN7wGul8S58Ho_4
title: CLPL Contract 2, id: 1LIe2-mRgohEUvkeDRkliDU0hOHLx_PNRzekAB_lcYMY
title: State of the Data 2, id: 12tp9L6SZlVRmeIKCwfo3PjDvyqXNCJkL2tiSO9kls8I
title: Singtel Data Scientist Cover Letter, id: 1Zfh0z9bjWlxFbwN0NvbskJzGNI9hSTGiIHZHCMwx-WI
title: Consultant Agreement - CL and Shaik 2.docx, id: 1333kHUyQgVO0oLZhG06L9SXiuvlN88sg
title: Invoice No: 002, id: 1nH6Mrakv8DKTDGb4O5rS0PdwffV1lwb_h1oG-tjEJuE
title: Untitled document, id: 1lxUZiSj9yaokcUB-gQQx-Dq17zisqA_ZYdQR7s1pjnk
title: STATE OF THE DATA 2  Steps forward, id: 1BPcCUNqfYujhECTgjGazk

In [26]:
from pydrive2.drive import GoogleDrive
# Create GoogleDrive instance with authenticated GoogleAuth instance.
drive = GoogleDrive(gauth)
filename = 'SWTT'
mimetype = 'application/vnd.google-apps.folder'
# Query
query = {'q': f"title = '{filename}' and mimeType='{mimetype}'"}
# Get list of files that match against the query
files = drive.ListFile(query).GetList()
files

[GoogleDriveFile({'kind': 'drive#file', 'userPermission': {'id': 'me', 'type': 'user', 'role': 'writer', 'kind': 'drive#permission', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1YE66HuLqo89aB78_LLrX9PWerbDcQklz/permissions/me', 'etag': '"n-T8ntPzwoU9-xUAJjSVjGQ3Vo8"', 'pendingOwner': False}, 'selfLink': 'https://www.googleapis.com/drive/v2/files/1YE66HuLqo89aB78_LLrX9PWerbDcQklz', 'ownerNames': ['hellosmartworldtt'], 'lastModifyingUserName': 'hellosmartworldtt', 'editable': True, 'writersCanShare': True, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [{'selfLink': 'https://www.googleapis.com/drive/v2/files/1YE66HuLqo89aB78_LLrX9PWerbDcQklz/parents/1ua92mVSsUFOGXgv6efBRoLHh3eASmssw', 'id': '1ua92mVSsUFOGXgv6efBRoLHh3eASmssw', 'isRoot': False, 'kind': 'drive#parentReference', 'parentLink': 'https://www.googleapis.com/drive/v2/files/1ua92mVSsUFOGXgv6efBRoLHh3eASmssw'}], 'appDataContents': False, 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/a

In [28]:
files[0]

GoogleDriveFile({'kind': 'drive#file', 'userPermission': {'id': 'me', 'type': 'user', 'role': 'writer', 'kind': 'drive#permission', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1YE66HuLqo89aB78_LLrX9PWerbDcQklz/permissions/me', 'etag': '"n-T8ntPzwoU9-xUAJjSVjGQ3Vo8"', 'pendingOwner': False}, 'selfLink': 'https://www.googleapis.com/drive/v2/files/1YE66HuLqo89aB78_LLrX9PWerbDcQklz', 'ownerNames': ['hellosmartworldtt'], 'lastModifyingUserName': 'hellosmartworldtt', 'editable': True, 'writersCanShare': True, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [{'selfLink': 'https://www.googleapis.com/drive/v2/files/1YE66HuLqo89aB78_LLrX9PWerbDcQklz/parents/1ua92mVSsUFOGXgv6efBRoLHh3eASmssw', 'id': '1ua92mVSsUFOGXgv6efBRoLHh3eASmssw', 'isRoot': False, 'kind': 'drive#parentReference', 'parentLink': 'https://www.googleapis.com/drive/v2/files/1ua92mVSsUFOGXgv6efBRoLHh3eASmssw'}], 'appDataContents': False, 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/ap

## 2) service account

In [17]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

def login_with_service_account():
    """
    Google Drive service with a service account.
    note: for the service account to work, you need to share the folder or
    files with the service account email.

    :return: google auth
    """
    # Define the settings dict to use a service account
    # We also can use all options available for the settings dict like
    # oauth_scope,save_credentials,etc.
    settings = {
                "client_config_backend": "service",
                "service_config": {
                    "client_json_file_path": "service-secrets.json",
                }
            }
    # Create instance of GoogleAuth
    gauth = GoogleAuth(settings=settings)
    # Authenticate
    gauth.ServiceAuth()
    return gauth

gauth = login_with_service_account()
drive = GoogleDrive(gauth)

In [15]:
# Auto-iterate through all files that match this query[^2^][2]
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
    # Check if the file is a folder; folders have a specific MIME type
    if file1['mimeType'] == 'application/vnd.google-apps.folder':
        print('title: %s, id: %s' % (file1['title'], file1['id']))


## 3) Langchain

In [29]:
from __future__ import print_function
import io
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from oauth2client.service_account import ServiceAccountCredentials
from PyPDF2 import PdfReader
from gspread import authorize, worksheet

# Replace with your credentials file path
CREDENTIALS_FILE = 'credentials.json'

# Replace with your spreadsheet ID
SPREADSHEET_ID = 'YOUR_SPREADSHEET_ID'
# Replace with the sheet name where you want to write data
SHEET_NAME = 'Sheet1'

def download_file_from_drive(url):
  """Downloads a file from Google Drive using the provided URL."""
  # Extract file ID from the URL
  file_id = url.split('/')[-2]
  
  # Initialize Google Drive service
  scopes = ['https://www.googleapis.com/auth/drive.readonly']
  credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, scopes)
  service = build('drive', 'v3', credentials=credentials)
  
  # Download the file
  request = service.files().get_media(fileId=file_id)
  fh = io.BytesIO()
  downloader = MediaIoBaseDownload(fh, request)
  downloader.set_chunksize(8192)
  downloader.download_from_beginning()
  return fh.getvalue()

def ocr_pdf(pdf_content):
  """Performs OCR on the PDF content and returns the extracted text."""
  reader = PdfReader(io.BytesIO(pdf_content))
  text = ""
  for page in reader.pages:
    text += page.extract_text()
  return text

def update_spreadsheet(text):
  """Updates the Google Sheet with the extracted text."""
  gc = authorize(credentials)
  sh = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_NAME)
  # Assuming you want to write the text to the first column (A1)
  sh.update_cell(1, 1, text)

def main():
  """Main function to handle user input, download, OCR, and update."""
  # Get Google Drive link from user
  drive_link = input("Enter Google Drive link: ")
  
  # Download PDF content
  pdf_content = download_file_from_drive(drive_link)
  
  # Perform OCR
  text = ocr_pdf(pdf_content)
  
  # Update Google Sheet
  update_spreadsheet(text)
  
  print("Text extracted from PDF and written to Google Sheet!")

In [39]:
from __future__ import print_function
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.service_account import ServiceAccountCredentials

# Replace with your credentials file path
CREDENTIALS_FILE = 'smart-platform.json'

# Replace with the ID of the target folder
FOLDER_ID = '1nstotWI9LYvUamNw-NSew-jnVDD7VAaH'
# https://drive.google.com/drive/folders/1nstotWI9LYvUamNw-NSew-jnVDD7VAaH?usp=drive_link

def get_google_drive_service():
  """Initializes the Google Drive API service."""
  scopes = ['https://www.googleapis.com/auth/drive.readonly']
  credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, scopes)
  service = build('drive', 'v3', credentials=credentials)
  return service

def download_file(service, file_id, file_name):
    """Downloads the specified file from Google Drive."""
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(file_name, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Download {int(status.progress() * 100)}%")

def get_folder_contents(service, folder_id):
    """Retrieves a list of PDF files within the specified folder and subfolders."""
    all_files = []
    page_token = None
    while True:
        try:
            results = service.files().list(
                pageSize=1000,  # Fetch a larger batch for efficiency
                fields="nextPageToken, files(id, name, mimeType)",
                q=f"'{folder_id}' in parents and mimeType='application/pdf'",  # Filter for PDFs
                pageToken=page_token
            ).execute()

            all_files.extend(results.get('files', []))
            page_token = results.get('nextPageToken')
            if not page_token:
                break  # No more pages

        except HttpError as error:
            print(f"An error occurred: {error}")
            return None

    return all_files


def get_download_link(item):
  """Retrieves the download link for a file based on its mimeType."""
  if item['mimeType'].startswith('application/'):  # Check if it's a Google Doc
    return None  # Google Docs don't have direct download links
  else:
    return f"https://drive.google.com/uc?export=download&id={item['id']}"


def main():
    """Main function to download PDFs from the folder."""
    service = get_google_drive_service()
    folder_contents = get_folder_contents(service, FOLDER_ID)

    if folder_contents:
        for item in folder_contents:
            file_name = item['name']
            file_id = item['id']
            print(f"Downloading: {file_name}")
            download_file(service, file_id, file_name)
            print(f"Downloaded: {file_name}")  
    else:
        print("No PDF files found in the specified folder.")



In [40]:
main()

Downloading: partners - malacca - attractions/menara taming sari.pdf


FileNotFoundError: [Errno 2] No such file or directory: 'partners - malacca - attractions/menara taming sari.pdf'